In [37]:
import pandas as pd
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

import re
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()
%matplotlib inline

In [38]:
train_data = pd.read_csv("../data/external/train_data.csv", sep=';')
train_labels = pd.read_csv("../data/external/train_labels.csv", sep=';')
test_data = pd.read_csv("../data/external/test_data.csv", sep=';')

In [39]:
#Заполнение пропусков строкой 'None'
train_data = train_data.fillna('None')
test_data = test_data.fillna('None')

In [40]:
train_data['okpd2_or_additional_code'] = train_data[['okpd2_code', 'additional_code']].apply(lambda x: x[0] if x[1] == 'None' else x[1], axis=1)
test_data['okpd2_or_additional_code'] = test_data[['okpd2_code', 'additional_code']].apply(lambda x: x[0] if x[1] == 'None' else x[1], axis=1)

In [43]:
# Формирование описания тендера
train_data['text_description_tender'] = train_data['purchase_name'] + " " + train_data['lot_name'] + " " + train_data['okpd2_names'] + " " + train_data['additional_code_names'] + " " + train_data['item_descriptions']
test_data['text_description_tender'] = test_data['purchase_name'] + " " + test_data['lot_name'] + " " + test_data['okpd2_names'] + " " + test_data['additional_code_names'] + " " + test_data['item_descriptions']

In [44]:
train_data.shape

(778901, 14)

In [45]:
test_data.shape

(84098, 14)

In [46]:
# train_data = train_data[:10000]
# test_data = test_data[:10000]

In [47]:
mystopwords = stopwords.words('russian')
ru_words = re.compile("[А-Яа-я]+")

# функция фильтрующая текст от остальных символов
def words_only(text):
    return " ".join(ru_words.findall(text))

# функция лемматизации текста
# def lemmatize(text, mystem=m):
#     try:
#         return "".join(m.lemmatize(text)).strip()  
#     except:
#         return " "
    
def lemmatize(text):
    try:
        return " ".join(pymorphy2_analyzer.parse(word)[0].normal_form for word in text.split())
    except:
        return " "

# функция для удаления стоп-слов
def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

# функция для удаления дубликатов (для случая когда в text_description_tender попал одинаковый текст при объединении текста из ячеек)
def remove_word_duplicates(text):
    text_ = []
    for word in text.split():
        if word not in text_:
            text_.append(word)
    return ' '.join(text_)    

# итоговая функция обработки текста
def preprocess(text):
    return remove_word_duplicates(remove_stopwords(lemmatize(words_only(text.lower()))))

In [48]:
digits = re.compile("[0-9]+")
def digits_only(text):
    return int(''.join(digits.findall(text)))

In [49]:
%%time
# обработка текста
train_data['text_description_tender_stemmed'] = train_data.text_description_tender.apply(preprocess)
test_data['text_description_tender_stemmed'] = test_data.text_description_tender.apply(preprocess)

Wall time: 4h 51min 22s


In [50]:
%%time
# приведение pn_lot_anon к числовому виду, чтобы использовать в качестве уникального идентификатора
train_data['pn_lot_id'] = train_data.pn_lot_anon.apply(digits_only)
test_data['pn_lot_id'] = test_data.pn_lot_anon.apply(digits_only)

Wall time: 1.35 s


In [51]:
# создаем датасеты с описаниями тендеров и сохраняем в csv файлы для дальнейшего использования
train_data_desc_stemmed = train_data[['pn_lot_id', 'text_description_tender_stemmed']]
test_data_desc_stemmed = test_data[['pn_lot_id', 'text_description_tender_stemmed']]

In [52]:
train_data_desc_stemmed

,pn_lot_id,text_description_tender_stemmed
0,7031618,услуга проведение финансовый аудит
1,7808247,приобретение приз проведение конкурс вопрос из...
2,7009496,приобретение приз проведение мероприятие избир...
3,5938735,продление лицензия битрикс управление сайт веб...
4,9327348,приобретение компьютерный техника машина вычис...
...,...,...
778896,2387328,пми закупка эа нефть сырой наименование товар ...
778897,2416734,пми закупка эа нефть сырой наименование товар ...
778898,1607958,пми закупка эа нефть сырой наименование товар ...
778899,8820511,пми закупка эа нефть сырой наименование товар ...


In [53]:
test_data_desc_stemmed

,pn_lot_id,text_description_tender_stemmed
0,2889061,услуга проведение финансовый аудит
1,5769273,приобретение офисный деревянный мебель стол пи...
2,3605798,оказание услуга предоставление лицензия право ...
3,1215937,приобретение офисный металлический мебель проч...
4,5025054,приобретение многофункциональный устройство мф...
...,...,...
84093,5373821,право заключение договор выполнение строительн...
84094,8396902,право заключение договор выполнение строительн...
84095,7569089,запрос ценовый предложение электронный форма п...
84096,2403905,право заключение договор выполнение проектный ...


In [54]:
train_data_desc_stemmed.to_csv('../data/intermid/train_data_desc_stemmed.csv', index=False, encoding='utf-8')
test_data_desc_stemmed.to_csv('../data/intermid/test_data_desc_stemmed.csv', index=False, encoding='utf-8')